In [ ]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.utilities.data_transformation import split_dataset_by_repo_and_source, load_all_files

In [ ]:
from processing_pipeline.data_optimization.stage_cfg import S1QARelevance

current_stage_cfg = S1QARelevance

In [ ]:
in_dir = current_stage_cfg.data_dir
out_dir = AbsDirPath.O_S1_QA_RELEVANCE_CHECK
df = load_all_files(in_dir)

In [ ]:
df = df.drop(columns=[current_stage_cfg.get_column_name("prompt")])


In [ ]:
df.head()

In [ ]:
df.shape

## Join data from the current stage with the optimized dataset

In [ ]:
df_main = load_all_files(AbsDirPath.O_KEYWORDS_MATCHING)

In [ ]:
df_main.head()

In [ ]:
import pandas as pd

df2 = pd.merge(df_main, df, "left", current_stage_cfg.depends_on_fields)
df2.head()

## Filter based on stage results


In [ ]:
current_stage_cfg.get_boolean_field()

In [ ]:
## Individual for every stage (False or True)
df2 = df2[df2[current_stage_cfg.get_boolean_field()] == True]
df2.shape

## Prepare data for the next stage

In [ ]:
next_stage_cfg = current_stage_cfg.next_stage

In [ ]:
required_columns_for_the_next_stage_prompt = next_stage_cfg.depends_on_fields
default_columns = ["repo_id", "source"]
total_columns = required_columns_for_the_next_stage_prompt + default_columns
required_columns_for_the_next_stage_prompt, total_columns

In [ ]:
df3 = df2.drop_duplicates(required_columns_for_the_next_stage_prompt)[total_columns]

In [ ]:
df3.head()

In [ ]:
df3.shape

In [ ]:
df3.groupby(["source"]).size()

## Save optimized data

In [ ]:
split_dataset_by_repo_and_source(out_dir, df3, clean_before_saving=True, drop_columns_before_save=default_columns)


In [ ]:
from utilities.split_parquet import split_files_exceeding_max_limit

split_files_exceeding_max_limit(out_dir)